# LOGISTIC REGRESSION USING AGE AND FARE

In [35]:
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
from pandas import get_dummies
import matplotlib as mpl
import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import warnings
import sklearn
import scipy
import numpy 
import json #EDA
import sys #EDA
import csv 
import os

In [36]:
df_train = pd.read_csv("C:/Users/SESA495280/Desktop/All training on Data Science/DATASETS/titanic/train.csv")
df_test = pd.read_csv("C:/Users/SESA495280/Desktop/All training on Data Science/DATASETS/titanic/test.csv")

In [37]:
df_train.shape

(891, 12)

In [38]:
df_test.shape

(418, 11)

In [39]:
type(df_train)

pandas.core.frame.DataFrame

In [40]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [41]:
##CHECK FOR NULL VALUES:
flag=df_train.isna().sum().any() ##CHECK FOR MISSING VALUES
flag

True

OR

In [42]:
##Check for missing values:
def check_missing_data(df):
    flag=df.isna().sum().any()
    if flag==True:
        total = df.isnull().sum() ##This line of code alone gives the number of missing values
        percent = (df.isnull().sum())/(df.isnull().count()*100)
        output = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
        data_type = []
        for col in df.columns:
            dtype = str(df[col].dtype)
            data_type.append(dtype)
        output['Types'] = data_type
        return(np.transpose(output))
    else:
        return(False)

In [43]:
check_missing_data(df_train)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Total,0,0,0,0,0,177,0,0,0,0,687,2
Percent,0,0,0,0,0,0.00198653,0,0,0,0,0.00771044,2.24467e-05
Types,int64,int64,int64,object,object,float64,int64,int64,object,float64,object,object


In [44]:
check_missing_data(df_test)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Total,0,0,0,0,86,0,0,0,1,327,0
Percent,0,0,0,0,0.00205742,0,0,0,2.39234e-05,0.00782297,0
Types,int64,int64,object,object,float64,int64,int64,object,float64,object,object


In [11]:
##TO DROP A ROW: df_train = df_train.dropna()
df_train_exp = df_train.copy()  ##We are creating a copy of the dataframe to ensure no changes happen on  the original dataframe
df_train_exp = df_train_exp.dropna()
check_missing_data(df_train_exp)

False

In [13]:
##To check unique values:
df_train["Pclass"].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [14]:
df_train["Cabin"].value_counts()

C23 C25 C27    4
B96 B98        4
G6             4
C22 C26        3
F33            3
              ..
E36            1
D30            1
C82            1
E46            1
C45            1
Name: Cabin, Length: 147, dtype: int64

In [15]:
#.cut from Pandas to create bin splits:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [50]:
##SUBSETTING VARIABLE (Considering only two variables)
df_train_dep = df_train['Survived']
df_train_ind = df_train[['Age','Fare']]

In [46]:
df_train_dep.shape

(891,)

In [47]:
df_train_ind.shape

(891, 2)

In [48]:
#Impute Age by mean
df_train.Age = df_train.Age.fillna(30)

In [31]:
df_test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.216507,0.447368,0.392344,35.627188
std,120.810458,0.841838,12.635016,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,23.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,30.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,35.750000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [49]:
#Impute Age by mean
df_test.Age = df_test.Age.fillna(30)

In [51]:
##TRAIN TEST SPLIT: 
## (INDEPENDENT VAR OF TRAIN & TEST, DEPENDENT VAR OF TRAIN & TEST)
X_train, X_test, y_train, y_test = train_test_split(df_train_ind,df_train_dep,test_size=0.3, random_state=100)  #random_state similar to set.seed() in R

In [52]:
##BUILDING A LOGISTIC REGRESSION
logreg1 = LogisticRegression()
logreg1.fit(X_train,y_train)             #.fit(indep,depen) : to apply any algorithm to data,model

C:\Users\SESA495280\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
##PREDICTING THE MODEL on TEST:
logreg_prediction = logreg1.predict(X_test)
##logreg_prediction

array([0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0], dtype=int64)

In [54]:
logreg_score=accuracy_score(y_test,logreg_prediction) ##Accuracy of LogReg = How many 0's are predicted as 0's and 1's as 1's
logreg_score                                    ##accuracy_score(actual_dependent variable,predicted values)

0.6529850746268657

In [55]:
##PREDICTING THE MODEL on TRAIN:
logreg_prediction_train = logreg1.predict(X_train)
logreg_prediction_train

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [56]:
logreg_score=accuracy_score(y_train,logreg_prediction_train) ##Accuracy of LogReg = How many 0's are predicted as 0's and 1's as 1's
logreg_score                                    ##accuracy_score(actual_dependent variable,predicted values)

0.6581059390048154

# DECISION TREE USING AGE AND FARE

In [58]:
from sklearn.tree import DecisionTreeClassifier

In [59]:
##Calling the function
DTree = DecisionTreeClassifier(random_state=1)

In [60]:
##Fitting the model:
DTree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1, splitter='best')

In [61]:
##Predicting the DTREE:
DTree_pred = DTree.predict(X_test)

In [62]:
##Calculating accuracy:
DTree_score=accuracy_score(y_test,DTree_pred)
DTree_score

0.6082089552238806

In [63]:
##Predicting the DTREE on the train data
DTree_pred_train = DTree.predict(X_train)
DTree_pred_train

array([0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,

In [64]:
##Calculating accuracy :
DTree_score=accuracy_score(y_train,DTree_pred_train)
DTree_score

0.9550561797752809

# RANDOM FOREST USING AGE AND FARE

In [65]:
rfc = RandomForestClassifier()

In [66]:
rfc.fit(X_train, y_train)

C:\Users\SESA495280\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [67]:
##PREDICTING THE MODEL on TEST:
rfc_prediction = rfc.predict(X_test)
rfc_prediction

array([1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0], dtype=int64)

In [68]:
##Calculating accuracy:
RFC_score=accuracy_score(y_test,rfc_prediction)
RFC_score

0.6119402985074627

In [69]:
##PREDICTING THE MODEL on TRAIN:
rfc_prediction2 = rfc.predict(X_train)
rfc_prediction2

array([0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,

In [73]:
##Calculating accuracy:
RFC_score2=accuracy_score(y_train,rfc_prediction2)
RFC_score2

0.9357945425361156

# RANDOM FOREST USING LOG OF AGE AND FARE

In [74]:
import math

In [75]:
##SUBSETTING VARIABLE:
train_log_ind = np.log(df_train[['Age','Fare']])

C:\Users\SESA495280\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


Above error is because if there are '0's then log 0 is infinity and undefined. So we replace or impute by 0.1

In [77]:
df_train['Fare']=df_train['Fare'].replace(0,0.1)

In [78]:
df_train['Fare'].describe()

count    891.000000
mean      32.205891
std       49.692338
min        0.100000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [79]:
df_train['Age'].describe()

count    891.000000
mean      29.758889
std       13.002570
min        0.420000
25%       22.000000
50%       30.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [85]:
##SUBSETTING VARIABLE:
train_log_ind = np.log(df_train[['Age','Fare']])
## We use df_train_dep = df_train['Survived'] for independent variable

In [82]:
df_test['Fare'].describe()

count    417.000000
mean      35.627188
std       55.907576
min        0.000000
25%        7.895800
50%       14.454200
75%       31.500000
max      512.329200
Name: Fare, dtype: float64

In [83]:
df_test['Fare']=df_test['Fare'].replace(0,0.1)

In [84]:
df_test['Fare'].describe()

count    417.000000
mean      35.627668
std       55.907270
min        0.100000
25%        7.895800
50%       14.454200
75%       31.500000
max      512.329200
Name: Fare, dtype: float64

In [86]:
##TRAIN TEST SPLIT: 
## (INDEPENDENT VAR OF TRAIN & TEST, DEPENDENT VAR OF TRAIN & TEST)
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(train_log_ind,df_train_dep,test_size=0.3, random_state=100)  #random_state similar to set.seed() in R

In [87]:
rfc = RandomForestClassifier()

In [88]:
rfc.fit(X_train_log, y_train_log)

C:\Users\SESA495280\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [89]:
##PREDICTING THE MODEL on TEST:
rfc_prediction3 = rfc.predict(X_test_log)
rfc_prediction3

array([1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0], dtype=int64)

In [90]:
##Calculating accuracy :
rfc_score3=accuracy_score(y_test_log,rfc_prediction3)
rfc_score3

0.6380597014925373

In [102]:
##PREDICTING THE MODEL on TRAIN:
rfc_prediction4 = rfc.predict(X_train_log)
#rfc_prediction4

In [92]:
##Calculating accuracy :
rfc_score4=accuracy_score(y_train_log,rfc_prediction4)
rfc_score4

0.9325842696629213

# RANDOM FOREST USING SQRT OF AGE AND FARE

In [95]:
##SUBSETTING VARIABLE:
train_sqrt_ind = np.sqrt(df_train[['Age','Fare']])

In [96]:
##TRAIN TEST SPLIT: 
## (INDEPENDENT VAR OF TRAIN & TEST, DEPENDENT VAR OF TRAIN & TEST)
X_train_sqrt, X_test_sqrt, y_train_sqrt, y_test_sqrt = train_test_split(train_sqrt_ind,df_train_dep,test_size=0.3, random_state=100)  #random_state similar to set.seed() in R

In [97]:
rfc = RandomForestClassifier()

In [98]:
rfc.fit(X_train_sqrt, y_train_sqrt)

C:\Users\SESA495280\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [101]:
##PREDICTING THE MODEL on TEST:
rfc_prediction5 = rfc.predict(X_test_sqrt)

In [100]:
##Calculating accuracy :
rfc_score5=accuracy_score(y_test_sqrt,rfc_prediction5)
rfc_score5

0.667910447761194

In [103]:
##PREDICTING THE MODEL on TRAIN:
rfc_prediction6 = rfc.predict(X_train_sqrt)
#rfc_prediction4

In [105]:
##Calculating accuracy :
rfc_score6=accuracy_score(y_train_sqrt,rfc_prediction6)
rfc_score6

0.9229534510433387

# RANDOM FOREST USING 'Sex', 'Parch', 'Pclass','Embarked','SibSp','Age','Fare'

In [106]:
##SUBSETTING VARIABLE:
## df_train_dep = df_train['Survived']
df_train_multi = df_train[['Age','Fare','Parch','SibSp','Embarked','Sex','Pclass']]

In [107]:
check_missing_data(df_train_multi)

,Age,Fare,Parch,SibSp,Embarked,Sex,Pclass
Total,0,0,0,0,2,0,0
Percent,0,0,0,0,2.24467e-05,0,0
Types,float64,float64,int64,int64,object,object,int64


In [108]:
df_train_multi.Embarked.describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [111]:
#Impute Age by MODE ('S') 
df_train_multi.Embarked = df_train_multi.Embarked.fillna('S')

In [119]:
def encode_features(df_train):

    features = ['Sex', 'Parch', 'Pclass','Embarked','SibSp','Age','Fare']

#     df_combined = pd.concat([df_train[features], df_test[features]])
    df_combined = df_train[features].copy()

   

    for feature in features:

        le = preprocessing.LabelEncoder()

        le = le.fit(df_combined[feature])

        df_train[feature] = le.transform(df_train[feature])

#         df_test[feature] = le.transform(df_test[feature])

    return df_train

In [120]:
df_train_enc = encode_features(df_train_multi)

C:\Users\SESA495280\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [116]:
df_train_enc

,Age,Fare,Parch,SibSp,Embarked,Sex,Pclass
0,28,18,0,1,2,1,2
1,51,207,0,1,0,0,0
2,34,41,0,0,2,0,2
3,47,189,0,1,2,0,0
4,47,43,0,0,2,1,2
...,...,...,...,...,...,...,...
886,35,85,0,0,2,1,1
887,24,153,0,0,2,0,0
888,39,131,2,1,2,0,2
889,34,153,0,0,0,1,0


In [121]:
##TRAIN TEST SPLIT: 
## (INDEPENDENT VAR OF TRAIN & TEST, DEPENDENT VAR OF TRAIN & TEST)
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(df_train_enc,df_train_dep,test_size=0.3, random_state=100)  #random_state similar to set.seed() in R

In [122]:
check_missing_data(df_train_enc)

False

In [123]:
check_missing_data(df_train_dep)

False

In [125]:
rfc_enc = RandomForestClassifier()

In [126]:
rfc_enc.fit(X_train_m, y_train_m)

C:\Users\SESA495280\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [127]:
##PREDICTING THE MODEL on TEST:
rfc_prediction6 = rfc_enc.predict(X_test_m)

In [128]:
##Calculating accuracy :
rfc_score6=accuracy_score(y_test_m,rfc_prediction6)
rfc_score6

0.7985074626865671

In [129]:
##PREDICTING THE MODEL on TRAIN:
rfc_prediction8 = rfc_enc.predict(X_train_m)
#rfc_prediction4

In [130]:
##Calculating accuracy :
rfc_score8=accuracy_score(y_train_m,rfc_prediction8)
rfc_score8

0.9759229534510433

# HYPER PARAMETER TUNING FOR A RANDOM FOREST

In [131]:
rfc_hpt = RandomForestClassifier()

In [132]:
parameters={'n_estimators':[4,6,9,11],
            'max_features':['log2','sqrt','auto'],
           'criterion':['entropy','gini'],
           'max_depth':[2,3,5,10],
           'min_samples_split':[2,3,5],
           'min_samples_leaf':[1,5,8]}

In [137]:
acc_score = make_scorer(accuracy_score) #This factory function wraps scoring functions for use in GridSearchCV and cross_val_score.

In [138]:
# RUN THE GRID SEARCH: Exhaustive search over specified parameter values for an estimator. Important members are fit, predict.
grid_obj = GridSearchCV(rfc_hpt, parameters, scoring = acc_score)
grid_obj = grid_obj.fit(df_train_enc,df_train_dep)

C:\Users\SESA495280\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [139]:
rfc_hpt = grid_obj.best_estimator_ # Set the clf to the BEST ESTIMATOR

In [140]:
rfc_hpt

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=11,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [144]:
#FITTING THE MODEL
rfc_hpt.fit(X_train_m, y_train_m)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=11,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [145]:
##PREDICTING THE MODEL on TEST:
rfc_pred = rfc_hpt.predict(X_test_m)

In [146]:
##Calculating accuracy :
rfc_best_score=accuracy_score(y_test_m,rfc_pred)
rfc_best_score

0.8171641791044776

In [150]:
##PREDICTING THE MODEL on TRAIN:
rfc_pred1 = rfc_hpt.predict(X_train_m)
#rfc_prediction4

In [151]:
##Calculating accuracy :
rfc_best_score1=accuracy_score(y_train_m,rfc_pred1)
rfc_best_score1

0.9181380417335474

# HYPER PARAMETER TUNING FOR A DECISION TREE

In [153]:
from sklearn.tree import DecisionTreeClassifier

In [158]:
##Calling the function
DTree1 = DecisionTreeClassifier(random_state=1)

In [159]:
parameters={'max_features':['log2','sqrt','auto'],
           'criterion':['entropy','gini'],
           'max_depth':[2,3,5,10],
           'min_samples_split':[2,3,5],
           'min_samples_leaf':[1,5,8]}

In [160]:
acc_score_dt = make_scorer(accuracy_score)

In [162]:
# RUN THE GRID SEARCH: Exhaustive search over specified parameter values for an estimator. Important members are fit, predict.
grid_obj = GridSearchCV(DTree1, parameters, scoring = acc_score_dt)
grid_obj = grid_obj.fit(df_train_enc,df_train_dep)

C:\Users\SESA495280\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [163]:
DTree1 = grid_obj.best_estimator_ # Set the clf to the BEST ESTIMATOR

In [164]:
#FITTING THE MODEL
DTree1.fit(X_train_m, y_train_m)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1, splitter='best')

In [165]:
##PREDICTING THE MODEL on TEST:
DT_pred = DTree1.predict(X_test_m)

In [166]:
##Calculating accuracy :
DT_score=accuracy_score(y_test_m,DT_pred)
DT_score

0.7761194029850746

In [167]:
##PREDICTING THE MODEL on TRAIN:
DT_pred1 = DTree1.predict(X_train_m)
#rfc_prediction4

In [169]:
##Calculating accuracy :
DT_score1=accuracy_score(y_train_m,DT_pred1)
DT_score1

0.8426966292134831

# XGBOOST

In [170]:
##Calling the function
xgboost = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)

In [171]:
xgboost.fit(X_train_m,y_train_m)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [172]:
##PREDICTING THE MODEL on TEST:
XGB_pred = xgboost.predict(X_test_m)

In [173]:
##Calculating accuracy :
XGB_score=accuracy_score(y_test_m,XGB_pred)
XGB_score

0.8171641791044776

In [174]:
##PREDICTING THE MODEL on TRAIN:
XGB_pred1 = xgboost.predict(X_train_m)
#rfc_prediction4

In [175]:
##Calculating accuracy :
XGB_score1=accuracy_score(y_train_m,XGB_pred1)
XGB_score1

0.8972712680577849

# XGBOOST AND HYPER PARAMETER TUNING

In [176]:
##Calling the function
xgboost = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)

In [188]:
parameters={'max_features':['log2','sqrt','auto'],
           'criterion':['entropy','gini'],
           'max_depth':[2,3,5,8],
           'min_samples_split':[2,3,5],
           'min_samples_leaf':[1,5,8]}

In [189]:
acc_score_xgb = make_scorer(accuracy_score)

In [190]:
# RUN THE GRID SEARCH: Exhaustive search over specified parameter values for an estimator. Important members are fit, predict.
grid_obj = GridSearchCV(xgboost, parameters, scoring = acc_score_xgb)
grid_obj = grid_obj.fit(df_train_enc,df_train_dep)

C:\Users\SESA495280\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [191]:
XGB1 = grid_obj.best_estimator_ # Set the clf to the BEST ESTIMATOR

In [192]:
#FITTING THE MODEL
XGB1.fit(X_train_m, y_train_m)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, criterion='entropy',
              gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=5,
              max_features='log2', min_child_weight=1, min_samples_leaf=1,
              min_samples_split=2, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [193]:
##PREDICTING THE MODEL on TEST:
XGB1_pred = XGB1.predict(X_test_m)

In [183]:
##Calculating accuracy :
XGB1_score=accuracy_score(y_test_m,XGB1_pred)
XGB1_score

0.8059701492537313

In [186]:
##PREDICTING THE MODEL on TRAIN:
XGB1_pred1 = XGB1.predict(X_train_m)
#rfc_prediction4

In [187]:
##Calculating accuracy :
XGB1_score1=accuracy_score(y_train_m,XGB1_pred1)
XGB1_score1

0.9406099518459069